In [1]:
!pip install torchsummary torch torchvision pandas torchaudio numpy librosa

In [2]:
import torch
from torchsummary import summary
from torch import nn
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision
import random
import pandas as pd
import torchaudio
import os
from os import walk
import numpy as np
from math import inf
import librosa
import librosa.feature
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score

if torch.cuda.is_available():
    DEVICE = "cuda:0"
else:
    DEVICE = "cpu"
print(f"Using device {DEVICE}.")

Using device cuda:0.


In [3]:
!if [ ! -d "UrbanSound8K" ]; then \
    sudo apt-get update && \
    sudo apt-get install -y aria2 && \
    aria2c -x 8 -s 8 -d . -o UrbanSound8K.tar.gz "https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz" && \
    tar -xvzf UrbanSound8K.tar.gz; \
fi

def parse_urbansound8k():
    dataset = pd.read_csv(f"UrbanSound8K/metadata/UrbanSound8K.csv")
    class_mapping = {
        0: "casual",
        1: "gunshots",
        2: "explosion",
        3: "siren",
        4: "fireworks",
    }

    mapping = {
        0: 0,
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0,
        6: 1,
        7: 0,
        8: 3,
        9: 0
    }

    if dataset["classID"][0].item() != 0:
        dataset["classID"] = dataset["classID"].map(mapping)
        dataset["class"] = dataset["classID"].map(class_mapping)
        dataset.to_csv("UrbanSound8K/metadata/UrbanSound8K.csv", index=False)

parse_urbansound8k()

Streaming output truncated to the last 5000 lines.
UrbanSound8K/audio/fold4/17480-2-0-6.wav
UrbanSound8K/audio/fold4/17480-2-0-9.wav
UrbanSound8K/audio/fold4/175904-2-0-11.wav
UrbanSound8K/audio/fold4/175904-2-0-24.wav
UrbanSound8K/audio/fold4/176003-1-0-0.wav
UrbanSound8K/audio/fold4/176638-5-0-0.wav
UrbanSound8K/audio/fold4/177756-2-0-10.wav
UrbanSound8K/audio/fold4/177756-2-0-4.wav
UrbanSound8K/audio/fold4/177756-2-0-5.wav
UrbanSound8K/audio/fold4/177756-2-0-7.wav
UrbanSound8K/audio/fold4/179862-1-0-0.wav
UrbanSound8K/audio/fold4/180977-3-1-1.wav
UrbanSound8K/audio/fold4/180977-3-1-5.wav
UrbanSound8K/audio/fold4/183989-3-1-21.wav
UrbanSound8K/audio/fold4/183989-3-1-23.wav
UrbanSound8K/audio/fold4/185709-0-0-0.wav
UrbanSound8K/audio/fold4/185709-0-0-1.wav
UrbanSound8K/audio/fold4/185709-0-0-6.wav
UrbanSound8K/audio/fold4/185709-0-0-7.wav
UrbanSound8K/audio/fold4/185909-2-0-102.wav
UrbanSound8K/audio/fold4/185909-2-0-13.wav
UrbanSound8K/audio/fold4/185909-2-0-17.wav
UrbanSound8K/audio

In [4]:
!if [ ! -d "SESA" ]; then \
    sudo apt-get update && \
    sudo apt-get install -y aria2 zip unzip && \
    aria2c -x 8 -s 8 -d . -o SESA.zip "https://zenodo.org/records/3519845/files/SESA.zip?download=1" && \
    unzip SESA.zip; \
fi

def parse_sesa(dir):
    SOUNDEVENTS = f"./SESA/{dir}/"
    filenames = next(walk(SOUNDEVENTS), (None, None, []))[2]
    filenames = [filename for filename in filenames if filename.strip()]

    mapping = {"casual": 0, "gunshot": 1, "explosion": 2, "siren": 3}

    mapping_reverse = ["casual", "gunshot", "explosion", "siren"]

    def get_class_ids(filenames):
        filenames_components = [filename.split("_") for filename in filenames]
        return [mapping[filename[0]] for filename in filenames_components]

    data = pd.DataFrame(
        data={
            "filename": filenames,
            "directory": dir,
            "id": [
                str(i).zfill(len(str(len(filenames))))
                for i in range(len(filenames))
            ],
            "class_id": get_class_ids(filenames),
            "class": [
                mapping_reverse[class_id] for class_id in get_class_ids(filenames)
            ],
        }
    )

    os.makedirs("./SESA/metadata", exist_ok=True)
    data.to_csv(f"./SESA/metadata/{dir}.csv", index=False)

if not os.path.exists("./SESA/metadata"):
    for dir in next(os.walk('./SESA'))[1]:
        if dir != "metadata":
            parse_sesa(dir)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [5]:
!if [ ! -d "ESC-50" ]; then \
    sudo apt-get update && \
    sudo apt-get install -y aria2 zip unzip  && \
    aria2c -x 8 -s 8 -d . -o ESC-50.zip "https://github.com/karoldvl/ESC-50/archive/master.zip" && \
    unzip ESC-50.zip && \
    mv ESC-50-master ESC-50; \
fi

def parse_esc50():
    dataset = pd.read_csv("ESC-50/meta/esc50.csv")

    class_mapping = {
        0: "casual",
        1: "gunshots",
        2: "explosion",
        3: "siren",
        4: "fireworks",
    }

    mapping = {
        42: 3,
        48: 4,
        19: 2,
    }

    for i in range(50):
        if i != 42 and i != 48 and i != 19:
            mapping[i] = 0

    if dataset["target"][1].item() != 0:
        dataset["target"] = dataset["target"].map(mapping)
        dataset["category"] = dataset["target"].map(class_mapping)
        dataset.to_csv("ESC-50/meta/esc50.csv", index=False)

parse_esc50()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [6]:
!if [ ! -d "gunshots" ]; then \
    sudo apt-get update && \
    sudo apt-get install -y aria2 zip unzip  && \
    aria2c -x 8 -s 8 -d . -o gunshots.zip "https://www.kaggle.com/api/v1/datasets/download/emrahaydemr/gunshot-audio-dataset" && \
    unzip gunshots.zip -d "gunshots"; \
fi

def parse_gunshots():
    filenames = []
    dirs = []
    for dir in next(os.walk('gunshots'))[1]:
        if dir != "metadata":
            L = next(walk(f"gunshots/{dir}"), (None, None, []))[2]
            L = [filename for filename in L if filename.strip()]
            filenames.extend(L)
            dirs.extend([dir] * len(L))

    data = pd.DataFrame(
        data={
            "filename": filenames,
            "directory": "gunshots",
            "subdirectory": dirs,
            "id": [
                str(i).zfill(len(str(len(filenames))))
                for i in range(len(filenames))
            ],
            "class_id": 1,
            "class": "gunshots",
        }
    )

    os.makedirs("./gunshots/metadata", exist_ok=True)
    data.to_csv(f"./gunshots/metadata/gunshots.csv", index=False)

if not os.path.exists("./gunshots/metadata"):
    parse_gunshots()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [7]:
!if [ ! -d "custom" ]; then \
    sudo apt-get update && \
    sudo apt-get install -y aria2 zip unzip  && \
    aria2c -x 8 -s 8 -d . -o custom.zip "https://www.dropbox.com/scl/fi/56zf96lhdlxqlttmawvvl/custom.zip?rlkey=541flpetiqxzic4ltxcvax8vz&st=inye9mny&dl=1" && \
    unzip custom.zip; \
fi

def parse_custom():
    filenames = []
    dirs = []
    classes = []
    for dir in next(os.walk('custom'))[1]:
        if dir != "metadata":
            L = next(walk(f"custom/{dir}"), (None, None, []))[2]
            L = [filename for filename in L if filename.strip()]
            filenames.extend(L)
            dirs.extend([dir] * len(L))
            classes.extend([dir] * len(L))

    class_mapping = {
        "casual": 0,
        "gunshots": 1,
        "explosion": 2,
        "siren": 3,
        "fireworks": 4,
    }

    data = pd.DataFrame(
        data={
            "filename": filenames,
            "directory": "custom",
            "subdirectory": dirs,
            "id": [
                str(i).zfill(len(str(len(filenames))))
                for i in range(len(filenames))
            ],
            "class_id": [class_mapping.get(c, -1) for c in classes],
            "class": classes,
        }
    )

    os.makedirs("./custom/metadata", exist_ok=True)
    data.to_csv(f"./custom/metadata/custom.csv", index=False)

if not os.path.exists("./custom/metadata"):
    parse_custom()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [8]:
def resnet_transformation(sr):
    class SpectralContrastTransform(object):
        def __init__(self, sr):
            self.sample_rate = sr

        def __call__(self, signal):
            signal = signal.cpu().numpy().flatten() # Move signal to CPU before converting to numpy
            spectral_contrast = librosa.feature.spectral_contrast(
                y=signal,
                sr=self.sample_rate,
                n_fft=2048,
                hop_length=512,
                win_length=None,
                window="hann",
                center=True,
                pad_mode="constant",
                fmin=90,
                n_bands=7,
                quantile=0.02,
                linear=False,
            )
            normalized_spectral_contrast = spectral_contrast
            return torch.tensor(normalized_spectral_contrast, dtype=torch.float32).to(DEVICE)

    return torchvision.transforms.Compose(
        [
            torchaudio.transforms.MFCC(
                sample_rate=sr,
                n_mfcc=61,
                melkwargs={"n_fft": 475, "hop_length": 190, "n_mels": 64},
            ).to(DEVICE),
            torchaudio.transforms.ComputeDeltas().to(DEVICE),
            SpectralContrastTransform(sr),
            lambda t: torch.tile(t, (3, 28, 4)).to(DEVICE),
            torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]).to(DEVICE),
        ]
    )

def rnn_transformation(sr):
    return torchvision.transforms.Compose(
        [
            torchaudio.transforms.MFCC(
                sample_rate=sr,
                n_mfcc=21,
                melkwargs={"n_fft": 551, "hop_length": 220, "n_mels": 40},
            ).to(DEVICE),
            torchaudio.transforms.ComputeDeltas().to(DEVICE),
            torchvision.transforms.Lambda(lambda x: x.squeeze(0)),
            torchvision.transforms.Lambda(lambda x: x.permute(1, 0))
        ]
    )

In [9]:
CLASS_MAPPING = [
    "casual",
    "gunshots",
    "explosion",
    "siren",
    "fireworks",
]

In [10]:
class AnnotatedDataset(Dataset):
    def __init__(self, annotation_file, audio_dir, transformation, target_sample_rate, num_samples,
    audio_sample_path_getter, audio_sample_label_getter):
        self.annotations = pd.read_csv(annotation_file)
        self.audio_dir = audio_dir
        self.transformation = transformation
        self.target_sample_rate = target_sample_rate
        self.num_samples = num_samples
        self.audio_sample_path_getter = audio_sample_path_getter
        self.audio_sample_label_getter = audio_sample_label_getter

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        # load audio sample and label associated with certain index
        audio_sample_path = self._get_audio_sample_path(index)
        label = self._get_audio_sample_label(index)
        signal, sr = torchaudio.load(audio_sample_path)
        signal = signal.to(DEVICE)
        signal = self._resample(signal, sr)
        signal = self._mix_down(signal)
        signal = self._adjust_length(signal)
        signal = self.transformation(signal)
        return signal, label

    def _adjust_length(self, signal):
        length_signal = signal.shape[1]
        if length_signal > self.num_samples:
            signal = signal[:, :self.num_samples]
        elif length_signal < self.num_samples:
            num_missing_samples = self.num_samples - length_signal
            last_dim_padding = (0, num_missing_samples)
            signal = torch.nn.functional.pad(signal, last_dim_padding)
        return signal

    def _resample(self, signal, sr):
        if sr != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate).to(DEVICE)
            signal = resampler(signal)
        return signal

    def _mix_down(self, signal):
        if signal.shape[0] > 1:
            signal = torch.mean(signal, dim=0, keepdim=True)
        return signal

    def _get_audio_sample_path(self, index):
        return self.audio_sample_path_getter(self.annotations, index, self.audio_dir)

    def _get_audio_sample_label(self, index):
        return self.audio_sample_label_getter(self.annotations, index)

class CollectiveDataset(Dataset):
    def __init__(self, datasets):
        self.datasets = datasets
        self.lengths = [len(dataset) for dataset in datasets]
        self.cumulative_lengths = np.cumsum(self.lengths)

    def __len__(self):
        return self.cumulative_lengths[-1]

    def __getitem__(self, index):
        if index < self.cumulative_lengths[0]:
            return self.datasets[0][index]
        for i in range(1, len(self.cumulative_lengths)):
            if index < self.cumulative_lengths[i]:
                r = self.datasets[i][index - self.cumulative_lengths[i - 1]]
                return r
        raise ValueError("Invalid index")

class CollectiveBinaryDataset(Dataset):
    def __init__(self, datasets):
        self.datasets = datasets
        self.lengths = [len(dataset) for dataset in datasets]
        self.cumulative_lengths = np.cumsum(self.lengths)

    def __len__(self):
        return self.cumulative_lengths[-1]

    def __getitem__(self, index):
        if index < self.cumulative_lengths[0]:
            r = self.datasets[0][index]
            return (r[0], 1 if r[1] > 0 else 0)
        for i in range(1, len(self.cumulative_lengths)):
            if index < self.cumulative_lengths[i]:
                r = self.datasets[i][index - self.cumulative_lengths[i - 1]]
                return (r[0], 1 if r[1] > 0 else 0)
        raise ValueError("Invalid index")

class SafeDataset(Dataset):
    def __init__(self, collective_dataset):
        self.collective_dataset = collective_dataset
        self.dataset = []
        count = 0
        for idx, dataset in enumerate(self.collective_dataset.datasets):
                for i in range(len(dataset)):
                    if dataset._get_audio_sample_label(i) == 0:
                        self.dataset.append((idx, i))
                        count += 1
        self.len = count

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        item = self.dataset[index]
        return self.collective_dataset.datasets[item[0]][item[1]]

class UnsafeDataset(Dataset):
    def __init__(self, collective_dataset):
        self.collective_dataset = collective_dataset
        self.dataset = []
        count = 0
        for idx, dataset in enumerate(self.collective_dataset.datasets):
                for i in range(len(dataset)):
                    if dataset._get_audio_sample_label(i) != 0:
                        self.dataset.append((idx, i))
                        count += 1
        self.len = count

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        item = self.dataset[index]
        return self.collective_dataset.datasets[item[0]][item[1]]

In [11]:
AUDIO_LENGTH = 4
SAMPLE_RATE = 22050
NUM_SAMPLES = SAMPLE_RATE * AUDIO_LENGTH

def init_urbansound8k():
    def get_audio_sample_path(annotations, index, audio_dir):
        return os.path.join(audio_dir, f"fold{annotations.iloc[index, 5]}", annotations.iloc[index, 0])

    def get_audio_sample_label(annotations, index):
        return annotations.iloc[index, 6]
    ANNOTATIONS_FILE = "UrbanSound8K/metadata/UrbanSound8K.csv"
    AUDIO_DIR = "UrbanSound8K/audio"

    usd = AnnotatedDataset(ANNOTATIONS_FILE, AUDIO_DIR, torch.nn.Identity(), SAMPLE_RATE, NUM_SAMPLES, get_audio_sample_path, get_audio_sample_label)
    return usd

def init_sesa(flavor="train"):
    def get_audio_sample_path(annotations, index, audio_dir):
        return os.path.join(audio_dir, annotations.iloc[index, 0])

    def get_audio_sample_label(annotations, index):
        return annotations.iloc[index, 3]
    ANNOTATIONS_FILE = f"SESA/metadata/{flavor}.csv"
    AUDIO_DIR = f"SESA/{flavor}"

    sesa = AnnotatedDataset(ANNOTATIONS_FILE, AUDIO_DIR, torch.nn.Identity(), SAMPLE_RATE, NUM_SAMPLES, get_audio_sample_path, get_audio_sample_label)
    return sesa

def init_esc50():
    def get_audio_sample_path(annotations, index, audio_dir):
        return os.path.join(audio_dir, annotations.iloc[index, 0])

    def get_audio_sample_label(annotations, index):
        return annotations.iloc[index, 2]
    ANNOTATIONS_FILE = "ESC-50/meta/esc50.csv"
    AUDIO_DIR = "ESC-50/audio"

    esc50 = AnnotatedDataset(ANNOTATIONS_FILE, AUDIO_DIR, torch.nn.Identity(), SAMPLE_RATE, NUM_SAMPLES, get_audio_sample_path, get_audio_sample_label)
    return esc50

def init_gunshots():
    def get_audio_sample_path(annotations, index, audio_dir):
        return os.path.join(audio_dir, annotations.iloc[index, 2], annotations.iloc[index, 0])

    def get_audio_sample_label(annotations, index):
        return annotations.iloc[index, 4]
    ANNOTATIONS_FILE = "gunshots/metadata/gunshots.csv"
    AUDIO_DIR = "gunshots"

    gunshots = AnnotatedDataset(ANNOTATIONS_FILE, AUDIO_DIR, torch.nn.Identity(), SAMPLE_RATE, NUM_SAMPLES, get_audio_sample_path, get_audio_sample_label)
    return gunshots

def init_custom():
    def get_audio_sample_path(annotations, index, audio_dir):
        return os.path.join(audio_dir, annotations.iloc[index, 2], annotations.iloc[index, 0])

    def get_audio_sample_label(annotations, index):
        return annotations.iloc[index, 4]
    ANNOTATIONS_FILE = "custom/metadata/custom.csv"
    AUDIO_DIR = "custom"

    gunshots = AnnotatedDataset(ANNOTATIONS_FILE, AUDIO_DIR, torch.nn.Identity(), SAMPLE_RATE, NUM_SAMPLES, get_audio_sample_path, get_audio_sample_label)
    return gunshots

def init_collective(datasets=[init_urbansound8k(), init_sesa(), init_esc50(), init_gunshots(), init_custom()]):
    return CollectiveDataset(datasets)

def init_collective_binary(datasets=[init_urbansound8k(), init_sesa(), init_esc50(), init_gunshots(), init_custom()]):
    return CollectiveBinaryDataset(datasets)

def init_unsafe(collective=init_collective()):
    return UnsafeDataset(collective)

def init_safe(collective=init_collective()):
    return SafeDataset(collective)

def save_model(model, name):
    torch.save(model.state_dict(), name)

def load_model(model, name):
    if os.path.exists(name):
        model.load_state_dict(torch.load(name, map_location=torch.device('cpu')))
    return model

In [12]:
class SoundClassifierGRU(nn.Module):
    def __init__(self, input_size=21, hidden_size=128, output_size=2, num_layers=4):
        super(SoundClassifierGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(
            input_size=input_size,  # Number of MFCC coefficients
            hidden_size=hidden_size,  # Size of the hidden state
            num_layers=num_layers,  # Number of stacked GRU layers
            batch_first=True  # Input shape is (batch_size, seq_len, input_size)
        )
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size, device=x.device)
        x, hidden = self.gru(x, hidden)
        x = x[:, -1, :]
        x = self.layer_norm(x)
        x = self.fc(x)
        return x


def init_rnn(name="gru.pth", load=True):
    model = SoundClassifierGRU()
    if load:
        load_model(model, name)
    return model.to(DEVICE)

def init_resnet(name="resnet.pth", load=True):
    model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT).to(DEVICE)
    model.fc = nn.Linear(model.fc.in_features, len(CLASS_MAPPING))
    if load:
        load_model(model, name)
    return model.to(DEVICE)

In [14]:
from google.colab import drive
drive.mount('/content/drive')
import locale
locale.getpreferredencoding = lambda: "UTF-8"
! cp "/content/drive/MyDrive/resnet.pth" "/content/"
! cp "/content/drive/MyDrive/gru.pth" "/content/"

Mounted at /content/drive


In [15]:
rnn = init_rnn()
resnet = init_resnet()

<ipython-input-11-a43f5b7d5169>:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(name, map_location=torch.device('cpu')))
Downloading: "htt

In [16]:
def evaluate_architechture(rnn, resnet, dataset):
    rnn.eval()
    resnet.eval()
    SAMPLE_RATE = 22050

    correct = 0
    total = 0
    with torch.no_grad():
        for i in range(len(dataset)):
            total += 1
            input, target = dataset[i][0].to(DEVICE), dataset[i][1]
            out_rnn = rnn(rnn_transformation(SAMPLE_RATE)(input).unsqueeze(0))[0].argmax(0)
            if out_rnn > 0:
                try:
                  out_resnet = resnet(resnet_transformation(SAMPLE_RATE)(input).unsqueeze(0))[0].argmax(0)
                  if CLASS_MAPPING[out_resnet] == CLASS_MAPPING[target]:
                      correct += 1
                except RuntimeError:
                  continue
            else:
              if CLASS_MAPPING[target] == "casual":
                correct += 1

    accuracy = correct / total * 100
    return accuracy

In [17]:
dataset = init_collective()
sesa_test = init_sesa("test")
train_accuracy = evaluate_architechture(rnn, resnet, dataset)
test_accuracy = evaluate_architechture(rnn, resnet, sesa_test)
print(f"Training Accuracy: {train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")

Training Accuracy: 88.75%
Test Accuracy: 40.95%
